In [206]:
import os
import datetime
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import glob
import scipy.io as sio

import netCDF4 as nc
from salishsea_tools import tidetools
from nowcast import (analyze, figures, research_ferries_test as rft)

%matplotlib inline

In [2]:
from importlib import reload

In [385]:
today = datetime.datetime.today()
yesterday = today - datetime.timedelta(days=12)
tomorrow = today + datetime.timedelta(days=1)
period = '1h'
grid = 'grid_T'
dmy = '07dec15'
date = datetime.datetime.strptime(dmy, "%d%b%y")
print(date)

2015-12-07 00:00:00


In [379]:
date = datetime.datetime.strptime(dmy, "%d%b%y")
date.strftime('%Y%m%d')

'20151208'

Step 0: Load the datasets and files that are available in nowcast/make_plots 
These will be used as inputs, not to be reloaded in the module)

In [357]:
grid_T = nc.Dataset(analyze.get_filenames(yesterday, tomorrow, period, grid, '/results/SalishSea/nowcast/')[0])
coastline = sio.loadmat('/ocean/rich/more/mmapbase/bcgeo/PNW.mat')
bathy, X, Y = tidetools.get_SS2_bathy_data()

## A : Preparing ferry data
Step 1: Get ferry data

In [383]:
reload(rft)

<module 'nowcast.research_ferries_test' from '/ocean/mdunn/MEOPAR/tools/SalishSeaNowcast/nowcast/research_ferries_test.py'>

In [386]:
obs = rft._get_sal_data('TWDP', dmy)

Step 2: Slicing ferry data

In [368]:
route_name='TWDP'
start_time = date.replace(hour=rft.route[route_name]['start']['hour'], minute=rft.route[route_name]['start']['minute'])
end_time = date.replace(hour=rft.route[route_name]['end']['hour'], minute=rft.route[route_name]['end']['minute'])
print(start_time)

2015-12-08 03:00:00


In [392]:
time_obs = rft.datenum2datetime(obs[0])
df = pd.DataFrame(time_obs)
j = np.logical_and(df>start_time, df<end_time)
j = np.array(j)
obs_route = obs[1:4,j]
obs_route[:,1:-1:20].shape


(3, 31)

## B: Preparing model data